<a href="https://colab.research.google.com/github/loisaida/LPPL/blob/master/Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
######################################################MOMENTUM FACTOR CALCULATION###########################
#IN
import pandas as pd


# Step 1: Read the initial rows to get the start and end dates
initial_rows = pd.read_csv('/content/single_hist_record.csv', nrows=2, header=None)

# Extract start and end dates
start_date = initial_rows.iloc[0, 1]
end_date = initial_rows.iloc[1, 1]

# Step 2: Read the main data, skipping the first three rows and setting 'ID' as the index
data_df = pd.read_csv('/content/single_hist_record.csv', skiprows=3, index_col='ID')

# Step 3: Read the reference file, setting 'ID' as the index
ref_df = pd.read_csv('/content/ig_esp_data_ref_3_6.csv', index_col='ID')

FileNotFoundError: [Errno 2] No such file or directory: '/content/single_hist_record.csv'

In [ ]:
########################calculates winners and losers
import pandas as pd
import numpy as np
import sqlite3

PERCENT_WINNER = 0.2  # Adjust between 0 and 1 as needed
PERCENT_LOSER = 0.2  # Adjust between 0 and 1 as needed

def flag_winners_losers(group_df, percent_winners, percent_losers):
    # Filter to include only quintiles 1 and 5
    quintiles_of_interest = group_df[group_df['DTS_Quintile'].isin([1, 5])]

    # Sort by 6m_total_return descending
    quintiles_of_interest = quintiles_of_interest.sort_values(by='6m_total_return', ascending=False)

    # Determine the indices for winners and losers based on the specified percentages
    num_entries = len(quintiles_of_interest)
    winners_cutoff = int(num_entries * percent_winners)
    losers_cutoff = int(num_entries * percent_losers)

    # Initialize flags
    quintiles_of_interest['IsWinner'] = 0
    quintiles_of_interest['IsLoser'] = 0

    # Apply flags
    quintiles_of_interest.iloc[:winners_cutoff, quintiles_of_interest.columns.get_loc('IsWinner')] = 1
    quintiles_of_interest.iloc[-losers_cutoff:, quintiles_of_interest.columns.get_loc('IsLoser')] = 1

    # Merge flags back to the main DataFrame based on ID
    group_df = group_df.merge(quintiles_of_interest[['ID', 'IsWinner', 'IsLoser']], on='ID', how='left').fillna({'IsWinner': 0, 'IsLoser': 0})

    return group_df

def process_multi_structure_data_to_db(filepath, db_path):
    with sqlite3.connect(db_path) as conn:
        # Your existing code for reading the data and iterating over date columns...

            if not group_df.empty:
                group_df['DTS'] = group_df['DURATION'] * group_df['SPREAD']
                group_df['DTS_Quintile'] = pd.qcut(group_df['DTS'], 5, labels=False) + 1

                # Assign DataFrameID
                group_df['DataFrameID'] = dataframe_id

                # Correctly flag winners and losers using the function
                group_df = flag_winners_losers(group_df, PERCENT_WINNER, PERCENT_LOSER)

                # Prepare DataFrame for insertion, ensuring column names match your SQLite table schema
                columns_for_db = ['DataFrameID', 'ID', 'DURATION', 'SPREAD', '1m_total_return', '6m_total_return', 'DTS', 'DTS_Quintile', 'IsWinner', 'IsLoser']
                group_df = group_df[columns_for_db]

                # Insert updated DataFrameEntries into the database
                group_df.to_sql('DataFrameEntries', conn, if_exists='append', index=False)

        conn.commit()

    return "Data processed and saved to database."

# Replace placeholders with actual paths
process_multi_structure_data_to_db('/content/hy-historical-return-data-static.csv', '/content/esp_hy_data_analysis.db')


In [1]:
############ calculates the mean of a mean of the 1_month_total_return for winners and losers for each month (DataFrame), and then displaying that in a table
import pandas as pd
import sqlite3

# Connect to the SQLite database
db_path = '/content/drive/MyDrive/database/esp_hy_data_analysis.db'
conn = sqlite3.connect(db_path)

# Fetch DataFrameIDs along with their corresponding start dates
query_dates = """
SELECT DataFrameID, StartDate
FROM DataFrames
ORDER BY StartDate
"""
df_dates = pd.read_sql_query(query_dates, conn)

# Initialize an empty DataFrame for the summary
summary_df = pd.DataFrame(columns=['date', 'winner_mean', 'loser_mean'])

for _, row in df_dates.iterrows():
    df_id = row['DataFrameID']
    start_date = row['StartDate']

    # Query to retrieve 1m_total_return, IsWinner, and IsLoser for the current DataFrameID
    query = f"""
    SELECT 1m_total_return, IsWinner, IsLoser
    FROM DataFrameEntries
    WHERE DataFrameID = {df_id}
    """
    data = pd.read_sql_query(query, conn)

    # Calculate mean of 1_month_total_return for winners and losers
    winner_mean = data[data['IsWinner'] == 1]['1m_total_return'].mean()
    loser_mean = data[data['IsLoser'] == 1]['1m_total_return'].mean()

    # Append the results to the summary DataFrame
    summary_df = summary_df.append({'date': start_date, 'winner_mean': winner_mean, 'loser_mean': loser_mean}, ignore_index=True)

# Close the database connection
conn.close()

# Display the summary DataFrame
print(summary_df)


OperationalError: unable to open database file

In [2]:
############### calculates descriptive statistics for DataFrame tables (months)
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# Path to your SQLite database
db_path = '/content/drive/MyDrive/database/esp_hy_data_analysis.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Get the list of unique DataFrameIDs representing each month's data
df_ids = pd.read_sql_query("SELECT DISTINCT DataFrameID FROM DataFrameEntries", conn)

# Initialize an empty DataFrame to hold descriptive statistics
stats_summary = pd.DataFrame()

for _, row in df_ids.iterrows():
    df_id = row['DataFrameID']

    # Query to retrieve 1_month_total_return for the current DataFrameID
    query = f"""
    SELECT 1m_total_return, IsWinner, IsLoser
    FROM DataFrameEntries
    WHERE DataFrameID = {df_id}
    """
    data = pd.read_sql_query(query, conn)

    # Calculate descriptive statistics for the entire set, winners, and losers
    desc_stats_entire = data['1m_total_return'].describe().rename('Entire Set')
    desc_stats_winners = data[data['IsWinner'] == 1]['1m_total_return'].describe().rename('Winners')
    desc_stats_losers = data[data['IsLoser'] == 1]['1m_total_return'].describe().rename('Losers')

    # Compile descriptive statistics into a DataFrame
    df_stats = pd.concat([desc_stats_entire, desc_stats_winners, desc_stats_losers], axis=1)
    df_stats['DataFrameID'] = df_id  # Add DataFrameID for reference

    # Append the stats of the current DataFrame to the summary DataFrame
    stats_summary = pd.concat([stats_summary, df_stats], axis=0)

    # Visualization (optional, as before)
    # ...

# Reset index of the summary DataFrame for better readability
stats_summary.reset_index(inplace=True)
stats_summary.rename(columns={'index': 'Statistic'}, inplace=True)

# Optionally, save the stats_summary DataFrame to a CSV file
stats_summary.to_csv('/content/drive/MyDrive/database/esp_hy_data_analysis_stats_summary.csv', index=False)

# Close the database connection
conn.close()

# Display the aggregate statistics across all DataFrames
aggregate_stats = stats_summary.groupby('Statistic').mean()  # Using mean here, but consider median or other aggregations depending on the data
print(aggregate_stats)
